In [12]:
import json
import requests

In [13]:
season = '2022'
league_id = '853208662'
url_template = f'https://fantasy.espn.com/apis/v3/games/flb/seasons/{season}/segments/0/leagues/{league_id}?view=mSettings&view=mRoster&view=mTeam&view=modular&view=mNav'

response = requests.get(url_template)

assert response.status_code == 200

In [14]:
obj = response.json()

teams = []
for team in obj['teams']:
  location = team['location']
  nickname = team['nickname']
  team_name = f'{location} {nickname}'

  roster = []
  for individual in team['roster']['entries']:
    player = individual['playerPoolEntry']['player']
    slots = player['eligibleSlots']

    ## bench, il
    slots.remove(16)
    slots.remove(17)

    roster.append({
      'id': player['id'],
      'name': player['fullName'],
      'slots': slots,
      'isPitcher': 13 in slots or 14 in slots or 15 in slots,
      'isBatter': any(filter(lambda x: x < 13, slots)),
    })

  teams.append({
    'name': team_name,
    'roster': roster,
  })

with open('../../data/mlb/roster.json', 'w') as output:
  output.write(
    json.dumps(teams, indent=3)
  )